#Chapter 24 RNN with PyTourch

In [1]:
import numpy as np
import torch
import torch.nn as nn

vocabSize = 20
featureSize = 10

W = torch.randn(vocabSize,featureSize)#creates matrixs 20x10
print(W.shape)

E = nn.Embedding.from_pretrained(W)#this embedding makes lookups faster.

idx = torch.LongTensor([1,2,5])
data = E(idx)
print(data)

E = nn.Embedding(vocabSize,featureSize)#random embeddings not fixed
data = E(idx)
print(data)


torch.Size([20, 10])
tensor([[ 6.8967e-01,  8.3772e-01, -1.4253e+00, -2.2678e+00,  1.5286e-01,
         -3.6639e-01,  3.8182e-01, -2.4628e-01,  4.3394e-01,  1.3959e+00],
        [ 4.4284e-01,  9.9146e-02, -1.7342e-03,  1.0122e+00,  5.2173e-01,
         -1.8305e-01, -1.3160e+00, -2.2298e-01, -1.2828e-02,  6.2070e-01],
        [ 2.5847e+00, -7.1213e-01,  3.7544e-01,  1.2159e+00, -5.1056e-02,
          3.9151e-01, -1.0942e-01, -8.8341e-01, -4.6353e-01,  1.4345e-01]])
tensor([[ 1.0976, -1.3319, -0.3476, -1.0160,  2.2616, -0.5990,  0.6283,  0.4848,
          0.1851,  1.7402],
        [-0.6835, -1.4256, -0.4018, -1.3473, -0.0106, -0.1029,  0.1179,  0.3500,
         -0.7413,  0.6821],
        [ 1.5700, -1.4144, -1.5505, -1.0328,  0.1556, -0.0759,  1.1951, -0.6270,
          0.5250, -1.3029]], grad_fn=<EmbeddingBackward0>)


In [2]:
vocabSize = 20
featureSize = 10

sqLength = 5#batch data in length of input words 5
hiddenSize = 2 # total number of nurions
numLayers = 1 # how many stack layers
batchSize = 4 # vocabSize / sqLength

idx = torch.LongTensor(np.arange(vocabSize))
E = nn.Embedding(vocabSize,featureSize)# 20 vectors and each vector will be 10 dimensional
data = E(idx)

inputs = data.view(batchSize,sqLength,featureSize)
print(inputs.shape)

rnn = nn.RNN(input_size=featureSize,hidden_size=hiddenSize,
             num_layers=numLayers,batch_first=True)

rnn_gru = nn.GRU(input_size=featureSize,hidden_size=hiddenSize,
             num_layers=numLayers,batch_first=True)

rnn_lstm = nn.LSTM(input_size=featureSize,hidden_size=hiddenSize,
             num_layers=numLayers,batch_first=True)

rnn_bi = nn.RNN(input_size=featureSize,hidden_size=hiddenSize,
             num_layers=3,batch_first=True,bidirectional=True)

#y_hat, h = rnn(inputs)
#print(y_hat)
y_hat, h = rnn_bi(inputs)
print(y_hat, h)
print(h.shape)

torch.Size([4, 5, 10])
tensor([[[ 0.4542, -0.8756,  0.8605, -0.8056],
         [ 0.6031, -0.8361, -0.4437,  0.5223],
         [ 0.9453, -0.9016,  0.6979, -0.6130],
         [ 0.8633, -0.6318, -0.0184,  0.4607],
         [ 0.4501, -0.7344,  0.6388, -0.0063]],

        [[ 0.3933, -0.2593, -0.6747,  0.8502],
         [ 0.6916, -0.9222,  0.8964, -0.8075],
         [ 0.6295, -0.2666, -0.2048,  0.6694],
         [ 0.7778, -0.7592, -0.0585,  0.1553],
         [ 0.5884, -0.8424,  0.7280, -0.2241]],

        [[ 0.1505, -0.7063,  0.6789, -0.5759],
         [ 0.7637, -0.5323, -0.7595,  0.8407],
         [ 0.5063, -0.9427,  0.8496, -0.6219],
         [ 0.7555, -0.9030,  0.5027, -0.2897],
         [ 0.6095, -0.8222,  0.0912, -0.0686]],

        [[-0.2271, -0.8925,  0.7370, -0.3857],
         [ 0.2084, -0.9297,  0.5034,  0.1139],
         [ 0.6838, -0.9451,  0.4152, -0.0731],
         [ 0.1723, -0.8817,  0.8375, -0.1438],
         [ 0.2235, -0.8790,  0.2395,  0.0550]]], grad_fn=<TransposeBackward1>)

In [3]:

print(y_hat[1,3,1])


outReshaped = y_hat.view(4,5,2,2)# batchsize,#seqlen,#directions,#hidden
y_hat_fwd = outReshaped[:,:,0,:]
y_hat_bck = outReshaped[:,:,1,:]
print(y_hat_fwd.shape)

print(h.shape)


hReshaped = h.view(3,2,4,2)#no layers,directions,batchsize, hidden
h_fwd = hReshaped[:,0,:,:]
h_bck = hReshaped[:,1,:,:]

print(h_fwd.shape)

y_hat,h = rnn_lstm(inputs)

print(y_hat.shape)


print(h[0].shape,h[1].shape)

y_hat,h = rnn(inputs)

print(y_hat.shape)

print(y_hat[1,3,1])

print(h.shape)


class RNN(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):# input_size = featuresize, hidden_size = No nurions
        super(RNN,self).__init__()
        self.hidden_size = hidden_size
        self.E = nn.Embedding(input_size,hidden_size)
        self.rnn = nn.GRU(hidden_size,hidden_size)
        self.linear = nn.Linear(hidden_size,output_size)
        self.sigmoid = nn.Sigmoid()
    def forward(self,input,hidden):
        emb = self.E(input).view(1,1,-1)
        output,hidden = self.rnn(emb,hidden)
        outLinear = self.linear(output)
        outSig = self.sigmoid(outLinear)
        return outSig,hidden

hidden_size = 128
model = RNN(20,hidden_size,1)


h = torch.zeros(1,1,hidden_size)
inputs = torch.LongTensor([2,3,4,8])
for input in inputs:
    y,h = model(input,h)

print(y)

tensor(-0.7592, grad_fn=<SelectBackward0>)
torch.Size([4, 5, 2])
torch.Size([6, 4, 2])
torch.Size([3, 4, 2])
torch.Size([4, 5, 2])
torch.Size([1, 4, 2]) torch.Size([1, 4, 2])
torch.Size([4, 5, 2])
tensor(0.9235, grad_fn=<SelectBackward0>)
torch.Size([1, 4, 2])
tensor([[[0.4429]]], grad_fn=<SigmoidBackward0>)
